
## 문제 1-1: 기본 체인 만들기 - AI 요리사

In [8]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence

# 환경 변수 로드 (.env에 OPENAI_API_KEY = gsk_... 형식으로 저장되어 있어야 함)
load_dotenv()

# 프롬프트 정의
prompt = PromptTemplate.from_template(
    """
당신은 친근하고 전문적인 AI 요리사입니다.
사용자가 제공한 재료로 만들 수 있는 요리를 추천하고,
간단한 레시피를 설명해주세요.

재료: {ingredients}

다음 형식으로 답변해주세요:
- 추천 요리명
- 필요한 재료 목록
- 조리법 (간단히 단계별로)

친근하고 따뜻한 말투로 설명해주세요.
"""
)

# Groq 기반 LLM 설정
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 출력 파서
parser = StrOutputParser()

# 체인 구성
chain: RunnableSequence = prompt | llm | parser

# 테스트 입력
user_input = "계란, 밥, 김치"

# 실행
result = chain.invoke({"ingredients": user_input})

# 출력
print("AI 요리사 추천 결과:")
print(result)


AI 요리사 추천 결과:
**김치볶음밥**

*   **필요한 재료 리스트** 계란, 밥, 김치, 고춧가루, 참기름, 소금, 후추

*   **조리법** 1.  김치를 잘게 썰어주세요. 2.  팬에 참기름을 두르고 썰어 놓은 김치를 넣고 볶아주세요. 3.  김치가 볶아지면 고춧가루를 넣어주세요. 4.  밥을 넣고 볶아주세요. 5.  소금과 후추로 간을 조절해 주세요. 6.  계란을 풀어주세요. 7.  팬에 기름을 두르고 계란을 넣고 섞어주세요. 8.  김치볶음밥과 함께 먹으면 더욱 맛있어요!


## 문제 1-2 : 2단계 체인 만들기 - 영화 추천 시스템 


In [10]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence

# 환경 변수 로드
load_dotenv()

# 모델 설정 (Groq 사용 가능)
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

parser = StrOutputParser()

# 1단계 체인: 장르 → 추천 영화
genre_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 장르 전문가입니다."),
    ("human", "{genre} 장르에서 추천할 만한 한국 또는 외국 영화를 1편만 추천하고, 한 줄로 간단히 설명해주세요.")
])
chain1 = genre_prompt | llm | parser

# 2단계 체인: 영화 제목 → 줄거리 요약
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 줄거리 요약 전문가입니다."),
    ("human", "{movie} 영화의 줄거리를 3줄로 요약해서 알려주세요.")
])
chain2 = summary_prompt | llm | parser

# 전체 파이프라인 체인
def full_chain(genre: str):
    # 1단계 실행
    recommended = chain1.invoke({"genre": genre})
    print("1단계 결과 - 추천 영화:")
    print(recommended.strip())

    # 영화 제목만 추출 (예: "미션 임파서블" 추천 → "미션 임파서블")
    movie = recommended.split("추천")[0].strip().replace(":", "").replace("는", "").strip("영화")
    
    # 2단계 실행
    summary = chain2.invoke({"movie": movie})
    print("\n2단계 결과 - 줄거리 요약:")
    print(f"영화: {movie}")
    print(summary.strip())

# 테스트 실행
user_input = "액션"
full_chain(user_input)


1단계 결과 - 추천 영화:
액션 장르에서 추천할 만한 영화는 **존 윅(John Wick, 2014)**입니다.

이 영화는 전직 암살자가 은퇴 후 평화로운 삶을 살던 중, 자신의 차를 도난당하고 개를 죽이는 사건이 발생하면서 벌어지는 액션으로 가득 찬 복수극입니다.

2단계 결과 - 줄거리 요약:
영화: 액션 장르에서
영화의 줄거리 요약은 다음과 같습니다.

경찰인 존은 마약조직의 보스를 체포하기 위해 잠복 수사를 시작합니다. 

존은 마약조직의 내부에 잠복하고 조직의 신뢰를 얻은 후, 보스를 체포하기 위한 증거를 수집합니다.

마침내 존은 보스를 체포하고 마약조직을 무너뜨리는 데 성공합니다.


## 문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용 

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 환경변수 로드
load_dotenv()

# 예시 데이터
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "테슬라는 자율주행 기술의 상용화를 앞당기기 위해 최신 버전의 FSD 소프트웨어를 공개했다. 이는 운전자의 개입을 최소화하면서도 안전성을 확보하는 데 중점을 두었다.",
        "keywords": "테슬라, 자율주행, FSD"
    }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

# FewShot Prompt 
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 최종 프롬프트 
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뉴스 키워드를 잘 뽑는 AI입니다. 주어진 뉴스에서 핵심 키워드 3개를 뽑아주세요."),
    few_shot_prompt,
    ("human", "{input}")
])

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 출력 파서
parser = StrOutputParser()

# 체인 연결
chain = final_prompt | llm | parser

# 테스트 입력
news_input = """제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 
모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 
내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."""

# 실행
result = chain.invoke({"input": news_input})

# 출력
print("키워드 추출 결과:")
print(result.strip())


키워드 추출 결과:
키워드: 제미나이, 구글, AI
